### Image Classification

1. Pull the raw image data from Flickr and create local copies of the images
2. Convert the images in to raw rgb arrays and labels
3. Label images

In [212]:
from os import listdir
from os.path import isfile, join

from PIL import Image
from scipy import misc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
import pymongo
import urllib

In [213]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [191]:
client = pymongo.MongoClient('localhost', 27017)
db = client.phronesis_food_photos

In [204]:
photos = db.remote_photos.find()
photodf = pd.DataFrame(list(photos))

In [205]:
photodf['label'] = photodf.title.apply(lambda t: t['_content'].lower())
photodf['url'] =  photodf.urls.apply(lambda u: u['url'][0]['_content'])
photodf['url'] = photodf.apply(create_download_link, 1)
photodf['timestamp'] = photodf.dates.apply(lambda u: pd.to_datetime(u['taken']))

In [206]:
pdf = photodf.loc[:, ['label', 'timestamp', 'url', 'farm', 'server', 'secret', 'id']]

In [211]:
# db.clean_photo_data.insert(pdf.to_dict(orient='records'))

In [190]:
def create_download_link(row):
    return "https://c2.staticflickr.com/%s/%s/%s_%s_q_d.jpg" % (row['farm'], row['server'], row['id'], row['secret'])

In [214]:
def save_image(row):
    template = "https://c2.staticflickr.com/%s/%s/%s_%s_q_d.jpg" % (row['farm'], row['server'], row['id'], row['secret'])
    filename = '../data/photos/%s.jpg' % row['id']
    urllib.urlretrieve(template, filename)

In [215]:
def img_to_matrix(filename, verbose=False):
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    img = list(img.getdata())
    img = map(list, img)
    img = np.array(img)
    return img

In [267]:
def show_image(img):
    img = np.swapaxes(img, 2, 0)
    plt.imshow(img)

In [219]:
# pdf.apply(save_image, 1)

In [75]:
path = '../data/'
filenmes = [f for f in listdir(path) if isfile(join(path, f))]

In [76]:
image_data = np.array([misc.imread('../data/' + img) for img in filenames])

In [109]:
swapped_image_data = np.swapaxes(image_data, 3, 1)

In [143]:
zipped_food_data = zip(swapped_image_data, pdf.title_text.as_matrix()[np.newaxis].T)

In [147]:
labeled_images = pdf[~pdf.title_text.isin(['', 'image'])]

In [150]:
labeled_ixs = labeled_images.index

In [163]:
clean_data = np.array(zipped_food_data)[labeled_ixs]

### Categorized Data

In [245]:
clean_photo_data = list(db.clean_photo_data.find({}, {'_id': 0}))

In [246]:
cdf = pd.DataFrame(clean_photo_data)

In [249]:
filenames = list(cdf[cdf.category.notnull()].id.apply(lambda f: '../data/photos/' + f + '.jpg' ))

In [250]:
image_data = np.array([misc.imread(img) for img in filenames])

In [251]:
swapped_image_data = np.swapaxes(image_data, 3, 1)

In [306]:
clean_data = np.array(zip(swapped_image_data, 
                          pd.factorize(cdf[cdf.category.notnull()].category.as_matrix())[0][np.newaxis].T))

### Data Prep

Data shape: (3-tensor of RGB data, label)

In [307]:
from sklearn.cross_validation import train_test_split

In [308]:
X = clean_data[:, 0]
y = clean_data[:, 1]

In [334]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### Keras 

In [280]:
# c10 = cifar10.load_data()

In [317]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

batch_size = 32
nb_epoch = 1
data_augmentation = False

### CIFAR

In [331]:
nb_classes = 10

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


### Food Data Version

In [332]:
nb_classes = cdf[cdf.category.notnull()].category.unique().shape[0]
# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB
img_channels = 3

# the data, shuffled and split between train and test sets
# (X_train, y_train), (X_test, y_test) = cifar10.load_data()

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

X_train shape: (50000, 3, 32, 32)
50000 train samples
10000 test samples


In [342]:
np.ravel(X_train).shape

(862,)

In [348]:
X_train[:2]

array([ array([[[189, 199, 204, ..., 213, 206, 200],
        [190, 199, 205, ..., 217, 213, 208],
        [191, 200, 205, ..., 212, 209, 207],
        ..., 
        [  3,   4,   5, ..., 235, 238, 241],
        [  8,   9,   9, ..., 235, 238, 241],
        [ 12,  13,  12, ..., 235, 239, 241]],

       [[164, 176, 184, ..., 202, 193, 183],
        [165, 176, 185, ..., 211, 204, 196],
        [166, 177, 186, ..., 210, 205, 199],
        ..., 
        [  7,   8,  10, ..., 203, 206, 209],
        [ 12,  13,  13, ..., 203, 206, 209],
        [ 16,  17,  16, ..., 204, 208, 210]],

       [[134, 145, 157, ..., 184, 174, 163],
        [134, 145, 158, ..., 195, 189, 180],
        [135, 146, 156, ..., 197, 193, 188],
        ..., 
        [  6,   7,   6, ..., 162, 165, 168],
        [ 13,  12,  12, ..., 162, 165, 168],
        [ 19,  18,  17, ..., 160, 164, 166]]], dtype=uint8),
       array([[[104, 135, 138, ...,  45,  49,  54],
        [118, 139, 140, ...,  47,  50,  53],
        [128, 138, 140,

In [319]:
# Create the model
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=(img_channels, img_rows, img_cols)))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [320]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [326]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

ValueError: setting an array element with a sequence.

In [316]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # this will do preprocessing and realtime data augmentation
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Using real-time data augmentation.
Epoch 1/1


KeyboardInterrupt: 

### Diagnostics

In [ ]:
# loss_and_metrics = model.evaluate(X_test, Y_test, batch_size=32)